In [70]:
import numpy as np
import random as rd
import math
import pandas as pd
eps = np.finfo(float).eps
from numpy import log2 as log

# class $Dtrees$ is used to Create the dataset for decision tree

In [71]:
class Dtrees:
    def __init__(self,k,m):
        self.k = k    #number of features
        self.m = m    #number of data points
        self.arrofarrX = []
        self.arrY = []
        
        for i in range(self.m):
            new = []
            tmpX = self.generateX()
            tmpW = self.generateW()
            tmpY = self.computeY(tmpX,tmpW)
            self.arrofarrX.append(tmpX)
            self.arrY.append(tmpY)
    
    def ran(self,x,y):
        if rd.uniform(x,y) > 0.5:
            return 1
        else:
            return 0
        
    def ran2(self,x,y):
        if rd.uniform(x,y) > 0.25:
            return 1
        else:
            return 0
        
    def generateX(self):
        new = []
        if self.ran(0,1) == 1:
            new.append(1)
        else:
            new.append(0)
        
        for x in range (1,self.k):
            if self.ran2(0,1) == 1:
                new.append(new[x-1])
            else:
                new.append(1-new[x-1])
        return new
    
    def generateW(self):
        new_w = []
        for x in range (1,self.k):
            de = 0
            for y in range(1,x+1):
                de = de+ math.pow(0.9,y)
            res = math.pow(0.9,x) / de
            new_w.append(res)
        return new_w
    
    def computeY(self,arrX,arrW):
        res = 0
        for x in range (1,self.k):
            res+= arrX[x]*arrW[x-1]
#         print("current result is %d",res)
        if res >= 0.5:
            return arrX[0]
        else:
            return 1 - arrX[0]
        

# Function used to Transfer data into data frame

In [338]:
def insertDS(ds,arr):
    tmp = len(arr)
    for x in range(tmp):
        ds[x] = arr[x]
    return ds

# Above

In [316]:
dataset = {'Taste':['Salty','Spicy','Spicy','Spicy','Spicy','Sweet','Salty','Sweet','Spicy','Salty'],
       'Temperature':['Hot','Hot','Hot','Cold','Hot','Cold','Cold','Hot','Cold','Hot'],
       'Texture':['Soft','Soft','Hard','Hard','Hard','Soft','Soft','Soft','Soft','Hard'],
       'Eat':['No','No','Yes','No','Yes','Yes','No','Yes','Yes','Yes']}

In [317]:

df = pd.DataFrame(dataset,columns=['Taste','Temperature','Texture','Eat'])

# START BUILDING DECISION TREE

# Compute $H(Y)$

In [437]:
d = Dtrees(4,20)
df = pd.DataFrame(d.arrofarrX)
L = len(d.arrofarrX[0])
df['Y'] = d.arrY

In [438]:
df

,0,1,2,3,Y
0,1,1,0,1,1
1,1,1,1,1,1
2,1,1,0,1,1
3,1,1,0,0,1
4,0,0,0,0,1
5,1,1,0,0,1
6,1,1,0,1,1
7,1,1,1,0,1
8,1,1,1,1,1
9,1,0,0,0,0


In [439]:
def find_entropy(df):
    ClFI = df.keys()[-1]   # to get the attribute name of classifcation column
    entropy = 0
    values = df[ClFI].unique()
    for value in values:
        fraction = df[ClFI].value_counts()[value]/len(df[ClFI])
        entropy += -fraction*np.log2(fraction)
    return entropy

# Compute $H(Y|X) = \sum_{x}P(X = x)[-\sum_{y}P(Y=y|X=x)\log P(Y=y|X=x)]$

In [440]:
def find_entropy_attribute(df,attribute):
    ClFI = df.keys()[-1]      
    target_variables = df[ClFI].unique()  
    variables = df[attribute].unique()    
    entropy2 = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df[ClFI] ==target_variable])
            den = len(df[attribute][df[attribute]==variable])
            fraction = num/(den+eps)
            entropy += -fraction*log(fraction+eps)
        fraction2 = den/len(df)
        entropy2 += -fraction2*entropy
    return abs(entropy2)

# Find the the attribute that could lead to the biggest entropy

In [441]:
def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
#         Entropy_att.append(find_entropy_attribute(df,key))
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)]

In [442]:
ans = find_winner(arr)

# get subtable since we find a attribute to cut the table

In [443]:
def get_subtable(df, node,value):
    return df[df[node] == value].reset_index(drop=True)

# (DFS IMPLEMENTATION)FINAL STEP: LETS BUILD DECISION TREE

In [450]:
def buildTree(df,tree=None): 
    Class = df.keys()[-1]   
    
    node = find_winner(df)   #get the attribute that could lead to he biggest entropy
    
    attValue = np.unique(df[node])   #get the distint attribute in the colomen which lead to the
                                     #biggest entrophy
    
    #init a dictonary structure to store the entire decision tree
    # which is a dictionary of dictionary
    # the innder dictionary used to store the current layer of decision tree
    if tree is None:                    
        tree={}
        tree[node] = {}
    
    
    #find the index of last column
    
    for value in attValue:
        
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable['Y'],return_counts=True)    #based on the subtable, if counts of unique array
                                                                        #equals one, means only one case exist to decide Y
        if len(counts)==1:
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) #DFS 
                   
    return tree

In [451]:
tree = buildTree(df)

In [452]:
import pprint
pprint.pprint(tree)

{3: {0: {2: {0: {0: {0: {1: {0: 1, 1: 0}}, 1: {1: {0: 0, 1: 1}}}}, 1: 1}},
     1: 1}}


In [ ]:
def predict(tree):
    

In [453]:
tree.keys()

dict_keys([3])